In [2]:
# import all tools needed
from bs4 import BeautifulSoup
import requests
import csv
import re
import pandas as pd

# Notes

**Wat wil ik doen?**  
Verzamel de contactinformatie voor elk van de 12 provincies in Nederland. De informatie staat op pagina's zoals deze: https://almanak.overheid.nl/14557/Provincie_Drenthe. Een overzicht van alle 12 pagina's staat hier: https://almanak.overheid.nl/Provincies/. 

  
**Hoe ga ik dit doen?**
1. Verzamel de 12 links naar provincie paginas
2. Voor elke pagina:
    - vraag de pagina op
    - maak een soepje
    - selecteer de benodigde informatie:
        - provincienaam
        - bezoekadres
        - postadres
        - telefoon
        - fax
        - website
        - email
    - verzamel de informatie in een variabele
3. Als dit werkt, dan maak ik een leeg CSV-bestand
4. Schrijf de informatie per pagina naar dit CSV-bestand
5. Scraper draaien, CSV bekijken. Klaar! :)

## 1. Verzamel de 12 links naar de provincie paginas

In [3]:
# verzamelen van de links van de overzichts pagina; stap 1 een request om de pagina op te vragen
r = requests.get('https://almanak.overheid.nl/Provincies/')

In [4]:
# als het gelukt is, dan is de status_code of response 200
r

<Response [200]>

In [5]:
# laat BeautifulSoup de HTML verwerken, zodat we die kunnen bevragen. Andersgezegd: maak een soepje
soup = BeautifulSoup(r.content, 'html.parser')

In [6]:
# check de soep
soup

<!DOCTYPE html>

<html lang="nl">
<head>
<title>Contactgegevens Provincies | Overheid.nl</title>
<link href="/static/onl/css/generic.css" rel="stylesheet" title="default"/>
<link href="/static/onl/images/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="https://almanak.overheid.nl/Provincies" name="DCTERMS.identifier" scheme="XSD.anyURI"/>
<meta content="Contactgegevens Provincies | Overheid.nl" name="DCTERMS.title"/>
<meta content="overzichtspagina" name="DCTERMS.type" scheme="OVERHEID.Informatietype"/>
<meta content="Het Register van Overheidsorganisaties bevat de namen, adressen en andere contactgegevens van de Nederlandse overheidsorganisaties. Ook de persoonsgegevens van (voornamelijk) leidinggevende personen binnen de Nederlandse overheid zijn opgenomen." name="DCTERMS.description"/>
<meta content="nl-N

In [7]:
# toon me alle links uit de soep
soup.find_all('a')

[<a href="https://www.overheid.nl/"><img alt="Overheid.nl" src="/static/onl/images/logo.svg"/></a>,
 <a href="#content">Direct naar de inhoud</a>,
 <a data-decorator="init-toggle-other-sites" data-handler="toggle-other-sites" href="#other-sites"><span class="visually-hidden">Andere sites binnen </span>Overheid.nl</a>,
 <a class="hidden-desktop" data-decorator="init-toggle-other-sites" data-handler="toggle-other-sites" href="#other-sites"><span class="visually-hidden">Andere sites binnen </span>Overheid.nl</a>,
 <a href="https://www.overheid.nl/aankondigingen-over-uw-buurt">Naar aankondigingen over uw buurt</a>,
 <a href="https://www.overheid.nl/dienstverlening">Naar dienstverlening</a>,
 <a href="https://www.overheid.nl/beleid-en-regelgeving">Naar beleid &amp; regelgeving</a>,
 <a href="/">Naar overheidsorganisaties</a>,
 <a href="https://www.overheid.nl/">Home</a>,
 <a href="/">Contactgegevens overheden</a>,
 <a href="/Provincies">Provincies</a>,
 <a href="/14557/Provincie_Drenthe">Pr

Omdat er veel links tussen zitten die we niet willen, ga ik de links filteren met een reguliere expressie. 
Alle links die ik wel wil beginnen met */ een getal / provincienaam*

In [8]:
# maak een lege lijst met provincie urls
urls = []

In [9]:
# voor elke link in alle links uit de soep...
for i in soup.find_all('a'):
    # als de link matcht met deze regex...
    if re.match(r'\/[0-9]+\/Provincie.*', i['href']):
        # voeg dan alleen de link, dus niet de hele 'a', 
        # maar alleen het url-deel: href=" WAT HIER STAAT " toe aan de lijst
        urls.append(i['href'])
    # als de link niet matcht...
    else:
        # doe niets en ga door naar de volgende
        pass

In [10]:
# len() geeft je de lengte van een lijst; als het goed is bevat urls 12 items
len(urls)

12

In [11]:
# dubbelcheck
urls

['/14557/Provincie_Drenthe',
 '/14866/Provincie_Flevoland',
 '/15184/Provincie_Fryslan',
 '/15185/Provincie_Gelderland',
 '/15632/Provincie_Groningen',
 '/16071/Provincie_Limburg',
 '/16317/Provincie_Noord-Brabant',
 '/16412/Provincie_Noord-Holland',
 '/16771/Provincie_Overijssel',
 '/17166/Provincie_Utrecht',
 '/18166/Provincie_Zeeland',
 '/17466/Provincie_Zuid-Holland']

## 2. Voor elke pagina...
**pagina opvragen**  

Om te testen gebruik ik alleen de eerste link uit de url-lijst. Vooraf heb ik wel alle links bekeken. Daardoor weet ik bijvoorbeeld dat niet alle tabellen een faxnummer bevatten...

In [12]:
# doe een requests, ipv de url zeg ik neem het 1e item uit de lijst urls
# een computer telt altijd vanaf 0, dus het 1e item is url[0]

# verder: je kunt teksten combineren door +
# dat de lijst dus maar de helft van de urls bevat los ik daarmee op

r = requests.get('https://almanak.overheid.nl'+urls[0])

In [13]:
# als het gelukt is, dan is de status_code of response 200
r

<Response [200]>

In [14]:
# laat BeautifulSoup de HTML verwerken, zodat we die kunnen bevragen. Andersgezegd: maak een soepje
soup = BeautifulSoup(r.content, 'html.parser')

**informatie selecteren**

In [15]:
# check de soep
soup

<!DOCTYPE html>

<html lang="nl">
<head>
<title>Contactgegevens Provincie Drenthe | Overheid.nl</title>
<link href="/static/onl/css/generic.css" rel="stylesheet" title="default"/>
<link href="/static/onl/images/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="https://almanak.overheid.nl/14557/Provincie_Drenthe" name="DCTERMS.identifier" scheme="XSD.anyURI"/>
<meta content="Contactgegevens Provincie Drenthe | Overheid.nl" name="DCTERMS.title"/>
<meta content="overzichtspagina" name="DCTERMS.type" scheme="OVERHEID.Informatietype"/>
<meta content="Het Register van Overheidsorganisaties bevat de namen, adressen en andere contactgegevens van de Nederlandse overheidsorganisaties. Ook de persoonsgegevens van (voornamelijk) leidinggevende personen binnen de Nederlandse overheid zijn opgenomen." name="DCTERMS.descrip

In [16]:
# alle informatie die ik wil staat in de 1e tabel, die ga ik er dus eerst uithalen
tabel1 = soup.find('table')

In [17]:
# check de tabel
tabel1

<table class="table__data-overview" data-roo-element="element-adresgegevens">
<tr>
<th scope="row">Bezoekadres</th>
<td data-before="Bezoekadres">
					Westerbrink 1<br/>
					9405 BJ Assen
				</td>
</tr>
<tr>
<th scope="row">Postadres</th>
<td data-before="Postadres">
					
					Postbus 122<br/>
					9400 AC Assen
				</td>
</tr>
<tr>
<th scope="row">Telefoon</th>
<td data-before="Telefoon">
						(0592) 36 55 55 (algemeen)
						
						
						
					</td>
</tr>
<tr>
<th scope="row">Fax</th>
<td data-before="Fax">(0592) 36 57 77</td>
</tr>
<tr>
<th scope="row">Internet</th>
<td data-before="Internet">
<a href="http://www.provincie.drenthe.nl">http://www.provincie.drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
<tr>
<th scope="row">E-mail</th>
<td data-before="E-mail">
<a href="mailto:post@drenthe.nl">post@drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
</table>

In [18]:
# het bezoekadres staat in de 1e td
bezoekadres = tabel1.find_all('td')[0]

In [19]:
# check het bezoekadres
bezoekadres

<td data-before="Bezoekadres">
					Westerbrink 1<br/>
					9405 BJ Assen
				</td>

In [20]:
# dat is meer dan we willen, geef alleen de tekst computer...
bezoekadres = tabel1.find_all('td')[0].get_text()

In [21]:
# controleer het bezoekadres
bezoekadres

'\n\t\t\t\t\tWesterbrink 1\n\t\t\t\t\t9405 BJ Assen\n\t\t\t\t'

In [22]:
# strip de tekst van onzin rondom het bezoekadres zelf, computer...
bezoekadres = tabel1.find_all('td')[0].get_text().strip()

In [23]:
# controleer het bezoekadres
bezoekadres

'Westerbrink 1\n\t\t\t\t\t9405 BJ Assen'

In [24]:
# beter, maar nog niet helemaal... laten we alle \t vervangen met niets
bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','')

In [25]:
# controleer het bezoekadres
bezoekadres

'Westerbrink 1\n9405 BJ Assen'

In [26]:
# als we de \n vervangen met een spatie, dan zijn we er wel...
bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','').replace('\n', ' ')

In [27]:
# controleer het bezoekadres
bezoekadres

'Westerbrink 1 9405 BJ Assen'

Ik wil het bezoekadres opsplitsen in straat + nummer; postcode; en plaats. Dat ga ik doen met behulp van `re.search(regex, input)` en `.group()`. Ik verdeel het bezoekadres (een groep in een regex staat tussen`(` en `)`) in 3 groepen (straat + nummer; postcode; plaats), en gebruik `.group()` om steeds 1 van die groepen te selecteren. 

De reguliere expressie die ik hiervoor gebruik ziet er zo uit: *(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)*

Je kunt dit testen door het bezoekadres 'Spiekersteeg 1 9461 BH GIETEN' in te vullen op [https://regex101.com](https://regex101.com).

Het adres deel van het bezoekadres ga ik opslaan onder variabele baAdres (bezoek adres adres).

In [28]:
# gebruik regex search om deze regex te vinden in het bezoekadres; geef daarna alleen de 1e groep terug.
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)

'Westerbrink 1'

In [29]:
# deze info opslaan naar variabele baAdres
baAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)

In [30]:
# check
baAdres

'Westerbrink 1'

Vooraf wist ik al dat ik niet alleen het adres van het bezoekadres apart wilde hebben, maar ook de postcode en de plaats. Daarom had ik in mijn regex direct 3 groepen gemaakt, *(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)*. 

Nu kan ik mijn regex-code dus hergebruiken, ik moet alleen het groep-nummer aanpassen.

In [31]:
# gebruik re.search om het bezoekadres te doorzoeken; selecteer de tweede groep
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)

'9405 BJ'

In [32]:
# postcode opslaan naar variabele
baPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)

In [33]:
# check
baPostcode

'9405 BJ'

In [34]:
# gebruik re.search om het bezoekadres te doorzoeken; selecteer de derde groep
re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)

'Assen'

In [35]:
# postcode opslaan naar variabele
baPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)

In [36]:
# check
baPlaats

'Assen'

Vervolgens doe ik precies hetzelfde voor het postadres: in tabel1 zoek ik naar naar de tweede *table data* (`.find_all('td')[1]`); en daarna splits ik het postadres op in paAdres, paPostcode en paPlaats (`re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres)`). 

Omdat ik wil dat de computer ook doorgaat als er iets niet lukt gebruik ik `try` en `except`. Daarmee zeg ik tegen de computer 'probeer dit te doen (opdracht na `try`), maar mocht dat nou niet lukken doe dan dit (alternatieve opdracht na `except`)'. 

Goed om te weten: `None` is computerjargon voor 'niets' of een lege cel. Door `None` te gebruiken weet ik zeker dat de computer het wel geprobeerd heeft en mijn programmaatje wel werkte, maar dat het om een of andere reden toch niet gelukt is.

In [37]:
# probeer het postadres uit tabel 1 te halen:
# navigeer naar 2e td, haal tekst eruit, strip spaties etc., vervang \t en \n
try:
    postadres = tabel1.find_all('td')[1].get_text().strip().replace('\t','').replace('\n', ' ')
except:
    postadres = None
# opsplitsen van postadres in paAdres, paPostcode en paPlaats
try:
    paAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(1)
except:
    paAdres = None
try:
    paPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(2)
except:
    paPostcode = None
try:
    paPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(3)
except:
    paPlaats = None

In [38]:
# te controle print ik de variablen...
print(postadres)
print(paAdres)
print(paPostcode)
print(paPlaats)

Postbus 122 9400 AC Assen
Postbus 122
9400 AC
Assen


Verder wil ik uit tabel1 nog het telefoonnummer en de site halen...

In [39]:
# probeer td 3 te vinden met het telefoonnummer in
# door try en except te gebruiken, gaat de computer verder ook als het niet lukt...
try:
    # ik strip het telefoonnummer meteen van alle niet haken en (algemeen) etc...
    telefoon = tabel1.find_all('td')[2].get_text().strip().replace(' (algemeen)', '').replace('(', '').replace(')','')
except:
    telefoon = None

In [40]:
# controle
print(telefoon)

0592 36 55 55


Niet alle provincies hebben een fax-nummer. Dus zeggen 'ga naar de vierde cel, en neem de inhoud daarvan, dat is het faxnummer', gaat niet werken. In sommige gevallen zal het kloppen, maar de kans bestaat dat het geen faxnummer is maar een website...

Daarom kunnen we dus niet navigeren door te tellen. Kijk naar de broncode om te zien hoe we wel kunnen navigeren: 

In [41]:
tabel1

<table class="table__data-overview" data-roo-element="element-adresgegevens">
<tr>
<th scope="row">Bezoekadres</th>
<td data-before="Bezoekadres">
					Westerbrink 1<br/>
					9405 BJ Assen
				</td>
</tr>
<tr>
<th scope="row">Postadres</th>
<td data-before="Postadres">
					
					Postbus 122<br/>
					9400 AC Assen
				</td>
</tr>
<tr>
<th scope="row">Telefoon</th>
<td data-before="Telefoon">
						(0592) 36 55 55 (algemeen)
						
						
						
					</td>
</tr>
<tr>
<th scope="row">Fax</th>
<td data-before="Fax">(0592) 36 57 77</td>
</tr>
<tr>
<th scope="row">Internet</th>
<td data-before="Internet">
<a href="http://www.provincie.drenthe.nl">http://www.provincie.drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
<tr>
<th scope="row">E-mail</th>
<td data-before="E-mail">
<a href="mailto:post@drenthe.nl">post@drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
</table>

Uniek voor de cel met het faxnummer er in, is `<td data-before="Fax">` voor het faxnummer. Bij geen enkele andere cel is de data-before 'Fax'. Een prima selectiecriterium dus.

In [42]:
# we zoeken nu alleen in tabel1 btw..
# verder zijn we op zoek naar een td
# we voegen alleen - in de vorm van een dictionary - de eigenschap toe waarop we willen selecteren
# een dictionary in Python ziet er zo uit: {"data-before": "Fax"}
tabel1.find('td', {"data-before": "Fax"})

<td data-before="Fax">(0592) 36 57 77</td>

In [43]:
# we willen natuurlijk alleen de tekst:
tabel1.find('td', {"data-before": "Fax"}).get_text()

'(0592) 36 57 77'

In [44]:
# laten we de haakjes eruit halen door ze te vervangen met niets
tabel1.find('td', {"data-before": "Fax"}).get_text().replace('(', '').replace(')','')

'0592 36 57 77'

In [45]:
# en nu nog even opslaan naar een variabele...
fax = tabel1.find('td', {"data-before": "Fax"}).get_text().replace('(', '').replace(')','')

Nu je weet hoe dit werkt, kun je dit natuurlijk ook gebruiken voor de site en mailadres. 
Je weet namelijk nooit zeker of dat de 5e, 6e of 7e cel is - dat is afhankelijk van een faxadres dat er
misschien wel, misschien niet in staat...


In [46]:
# even kijken wat er uniek is aan mailadres en website...
tabel1

<table class="table__data-overview" data-roo-element="element-adresgegevens">
<tr>
<th scope="row">Bezoekadres</th>
<td data-before="Bezoekadres">
					Westerbrink 1<br/>
					9405 BJ Assen
				</td>
</tr>
<tr>
<th scope="row">Postadres</th>
<td data-before="Postadres">
					
					Postbus 122<br/>
					9400 AC Assen
				</td>
</tr>
<tr>
<th scope="row">Telefoon</th>
<td data-before="Telefoon">
						(0592) 36 55 55 (algemeen)
						
						
						
					</td>
</tr>
<tr>
<th scope="row">Fax</th>
<td data-before="Fax">(0592) 36 57 77</td>
</tr>
<tr>
<th scope="row">Internet</th>
<td data-before="Internet">
<a href="http://www.provincie.drenthe.nl">http://www.provincie.drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
<tr>
<th scope="row">E-mail</th>
<td data-before="E-mail">
<a href="mailto:post@drenthe.nl">post@drenthe.nl</a>
						 (algemeen)
						
						
					</td>
</tr>
</table>

In [47]:
# we zoeken in tabel1 naar een td, met als data-before eigenschap E-mail
tabel1.find('td', {"data-before": "E-mail"})

<td data-before="E-mail">
<a href="mailto:post@drenthe.nl">post@drenthe.nl</a>
						 (algemeen)
						
						
					</td>

In [48]:
# we willen alleen de link
tabel1.find('td', {"data-before": "E-mail"}).find('a')

<a href="mailto:post@drenthe.nl">post@drenthe.nl</a>

In [49]:
# we willen natuurlijk alleen de link waar we naar verwezen worden als we op de link klikken
# deze link staat bij href=""; de klikbare tekst die je ziet (had ook 'mail naar Drenthe' kunnen zijn)
# staat tussen > en </a>; de klikbare tekst en de werkelijke link zijn niet altijd hetzelfde
# zoals hier het geval is, dus ga ik voor de zekerheid altijd voor de href-url
tabel1.find('td', {"data-before": "E-mail"}).find('a')['href']

'mailto:post@drenthe.nl'

In [50]:
# nu nog mailto: vervangen met niets...
tabel1.find('td', {"data-before": "E-mail"}).find('a')['href'].replace('mailto:', '')

'post@drenthe.nl'

In [51]:
# en opslaan naar een variabele 
email = tabel1.find('td', {"data-before": "E-mail"}).find('a')['href'].replace('mailto:', '')

In [52]:
# als je het opslaat als variabele, wordt er niets meer geprint. 
# als check zou je dit kunnen doen:
email

'post@drenthe.nl'

In [53]:
# laten we hetzelfde doen voor de website
tabel1.find('td', {"data-before": "Internet"})

<td data-before="Internet">
<a href="http://www.provincie.drenthe.nl">http://www.provincie.drenthe.nl</a>
						 (algemeen)
						
						
					</td>

In [54]:
# we willen alleen de link
tabel1.find('td', {"data-before": "Internet"}).find('a')

<a href="http://www.provincie.drenthe.nl">http://www.provincie.drenthe.nl</a>

In [55]:
# we willen natuurlijk alleen de link waar we naar verwezen worden als we op de link klikken
# deze link staat bij href=""; de klikbare tekst die je ziet (had ook 'mail naar Drenthe' kunnen zijn)
# staat tussen > en </a>; de klikbare tekst en de werkelijke link zijn niet altijd hetzelfde
# zoals hier het geval is, dus ga ik voor de zekerheid altijd voor de href-url
tabel1.find('td', {"data-before": "Internet"}).find('a')['href']

'http://www.provincie.drenthe.nl'

In [56]:
# en opslaan naar een variabele 
website = tabel1.find('td', {"data-before": "Internet"}).find('a')['href']

In [57]:
# als je het opslaat als variabele, wordt er niets meer geprint. 
# als check zou je dit kunnen doen:
website

'http://www.provincie.drenthe.nl'

We zouden zomaar het belangrijkste vergeten... Waar halen we de provincienaam vandaan? 

Ik ga met een regex de naam uit de url halen... Komtie:

In [58]:
# print een url zodat ik weet waar ik mee moet werken
urls[0]

'/14557/Provincie_Drenthe'

Op basis van wat ik hier zie, denk ik nu: selecteer alles na 'Provincie_'. Dat leidt tot de volgende regex 'Provincie\_(.*)'. Te testen op [regex101.com](http://regex101.com)

In [59]:
provincienaam = re.search(r'Provincie\_(.*)', urls[0]).group(1)

In [60]:
# print provincienaam als check
provincienaam

'Drenthe'

In [63]:
# check of het met alle urls werkt...
# voor elk item (i) in de lijst urls
for i in urls:
    # selecteer de provincienaam - omdat we niet meer testen gebruik ik i ipv urls[0]
    provincienaam = re.search(r'Provincie\_(.*)', i).group(1)
    # als check wil ik de provincienaam geprint hebben, dan kan ik zien of het werkt
    print(provincienaam)

Drenthe
Flevoland
Fryslan
Gelderland
Groningen
Limburg
Noord-Brabant
Noord-Holland
Overijssel
Utrecht
Zeeland
Zuid-Holland


Nu ga ik al deze onderdelen samenvoegen in een for-loop. Met deze for-loop zeg ik eigenlijk tegen de computer: voor elke link in het lijstje urls, vraag de pagina op, en verzamel al deze informatie.

Het grootste deel van de code hieronder heb je al eerder gezien, dus geen reden om te schrikken. :) 

In [64]:
# voor elk item (i) in de lijst urls... 
# let op, om te testen beperk ik de lijst tot de 1e twee items
# [:2] betekent alle items in de lijst tot (NIET tot en met, dus EXCLUSIEF) item 2
# omdat computer vanaf 0 tellen, levert dit 2 resultaten op: item 0 en item 1 exclusief item 2
for i in urls[:2]:
    # probeer...
    try:
        # de provincienaam uit de url te selecteren
        provincienaam = re.search(r'Provincie\_(.*)', i).group(1)
    # als dat niet lukt...
    except:
        # laat de variabele provincienaam dan leeg
        provincienaam = None
    # vraag de pagina op, let op de url maak ik ter plekke
    r = requests.get('https://almanak.overheid.nl'+i)
    # als het opvragen van de pagina gelukt is...
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        # probeer...
        try:
            #  ...de 1e tabel uit de soep te halen
            tabel1 = soup.find('table')
        # als dat niet lukt...
        except:
            # ...print dan de volgende foutmelding
            print('Tabel1 niet gevonden - dus niets zal gaan werken :(')
        # probeer...
        try:
            # ...het bezoekadres uit de tabel1 te halen
            bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','').replace('\n', ' ')
        # als dat niet lukt...
        except:
            # ...maak het bezoekadres dan 'None', which is programmers speak for leave it empty
            bezoekadres = None
        # probeer...
        try:
            # het bezoekadres op te splitsen in adres met een regex
            baAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)
        # als dat niet lukt...
        except:
            # ...maak het baAdres dan 'None'
            baAdres = None
        # probeer...
        try:
            # ...baPostcode te selecteren uit het bezoekadres met een regex
            baPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)
        # als dat niet lukt...
        except:
            # ...maak baPostcode dan None
            baPostcode = None
        # probeer...
        try:
            # ...baPlaats te selecteren uit het bezoekadres met een regex
            baPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)
        # als dat niet lukt...
        except:
            # ... maak baPlaats dan None
            baPlaats = None
        # herhaal bovenstaande, maar dan voor het postadres. Stap 1: selecter het postadres
        try:
            postadres = tabel1.find_all('td')[1].get_text().strip().replace('\t','').replace('\n', ' ')
        except:
            postadres = None
        # Stap 2: het opsplitsen van postadres in paAdres, paPostcode en paPlaats
        try:
            paAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(1)
        except:
            paAdres = None
        try:
            paPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(2)
        except:
            paPostcode = None
        try:
            paPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(3)
        except:
            paPlaats = None
        # probeer het telefoonnummer op te halen
        try:
            telefoon = tabel1.find_all('td')[2].get_text().strip().replace(' (algemeen)', '').replace('(', '').replace(')','')
        except:
            telefoon = None
        # probeer het faxnummer op te halen
        try:
            fax = tabel1.find('td', {"data-before": "Fax"}).get_text().replace('(', '').replace(')','')
        except:
            fax = None
        # probeer het mailadres op te halen
        try:
            email = tabel1.find('td', {"data-before": "E-mail"}).find('a')['href'].replace('mailto:', '')
        except:
            email = None
        # probeer de website op te halen
        try:
            website = tabel1.find('td', {"data-before": "Internet"}).find('a')['href']
        except:
            website = None
        # verzamel alle informatie in 1 variabele
        data = (provincienaam, baAdres, baPostcode, baPlaats, paAdres, paPostcode, paPlaats, telefoon, fax, email, website)
        print(data)      
    # als het opvragen van de pagina niet gelukt is...
    else:
        # print feedback zodat ik kan zien waar het fout ging
        print('https://almanak.overheid.nl'+i+' opvragen is niet gelukt...')

('Drenthe', 'Westerbrink 1', '9405 BJ', 'Assen', 'Postbus 122', '9400 AC', 'Assen', '0592 36 55 55', '0592 36 57 77', 'post@drenthe.nl', 'http://www.provincie.drenthe.nl')
('Flevoland', 'Visarenddreef 1', '8232 PH', 'Lelystad', 'Postbus 55', '8200 AB', 'Lelystad', '0320 265 265', '0320 26 52 60', 'provincie@flevoland.nl', 'http://www.flevoland.nl')


Het enige dat ik nu nog moet doen, is zorgen dat de resultaten naar een CSV-bestand worden geschreven. Daarvoor voeg ik aan het begin en aan het einde een klein beetje code toe. 

Hieronder het eindresultaat: een scraper die data opslaat in een csv. :) 

In [65]:
# maak een bestand met de naam '190723 contactgegevens provincies.csv'; maak het 'writeable' = 'w'; 
# zorg dat computers het kunnen lezen met encoding utf-8; en maak dit bestand aanroepbaar onder de naam/variabele f
f = open('190723 contactgegevens provincies.csv', 'w', encoding='utf8', newline='')
# maak een schrijver waarmee we informatie naar ons bestand, f, kunnen schrijven.
# omdat we een csv willen maken, een comma seperated values file, scheiden we waarden met de delimiter een komma. 
writer = csv.writer(f, delimiter=',')
# gebruik de zojuist gemaakte schrijver om de 1e rij met kolomhoofden naar het bestand te schrijven 
# let op de volgorde van de kolomhoofden moet overeenkomen met de volgorde van de weg te schrijven data, 
# zie de variabele 'data' onderaan deze scraper.
writer.writerow(['provincie', 'baAdres', 'baPostcode', 'baPlaats', 'paAdres', 'paPostcode', 
                 'paPlaats',  'telefoon', 'fax', 'email', 'website'])
# voor elk item (i) in de lijst urls... 
for i in urls:
    # probeer...
    try:
        # de provincienaam uit de url te selecteren
        provincienaam = re.search(r'Provincie\_(.*)', i).group(1)
    # als dat niet lukt...
    except:
        # laat de variabele provincienaam dan leeg
        provincienaam = None
    # vraag de pagina op, let op de url maak ik ter plekke
    r = requests.get('https://almanak.overheid.nl'+i)
    # als het opvragen van de pagina gelukt is...
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        # probeer...
        try:
            #  ...de 1e tabel uit de soep te halen
            tabel1 = soup.find('table')
        # als dat niet lukt...
        except:
            # ...print dan de volgende foutmelding
            print('Tabel1 niet gevonden - dus niets zal gaan werken :(')
        # probeer...
        try:
            # ...het bezoekadres uit de tabel1 te halen
            bezoekadres = tabel1.find_all('td')[0].get_text().strip().replace('\t','').replace('\n', ' ')
        # als dat niet lukt...
        except:
            # ...maak het bezoekadres dan 'None', which is programmers speak for leave it empty
            bezoekadres = None
        # probeer...
        try:
            # het bezoekadres op te splitsen in adres met een regex
            baAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(1)
        # als dat niet lukt...
        except:
            # ...maak het baAdres dan 'None'
            baAdres = None
        # probeer...
        try:
            # ...baPostcode te selecteren uit het bezoekadres met een regex
            baPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(2)
        # als dat niet lukt...
        except:
            # ...maak baPostcode dan None
            baPostcode = None
        # probeer...
        try:
            # ...baPlaats te selecteren uit het bezoekadres met een regex
            baPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', bezoekadres).group(3)
        # als dat niet lukt...
        except:
            # ... maak baPlaats dan None
            baPlaats = None
        # herhaal bovenstaande, maar dan voor het postadres. Stap 1: selecter het postadres
        try:
            postadres = tabel1.find_all('td')[1].get_text().strip().replace('\t','').replace('\n', ' ')
        except:
            postadres = None
        # Stap 2: het opsplitsen van postadres in paAdres, paPostcode en paPlaats
        try:
            paAdres = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(1)
        except:
            paAdres = None
        try:
            paPostcode = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(2)
        except:
            paPostcode = None
        try:
            paPlaats = re.search(r'(.*)\s([0-9]{4,4}\s[A-Z]+)\s(.*)', postadres).group(3)
        except:
            paPlaats = None
        # probeer het telefoonnummer op te halen
        try:
            telefoon = tabel1.find_all('td')[2].get_text().strip().replace(' (algemeen)', '').replace('(', '').replace(')','')
        except:
            telefoon = None
        # probeer het faxnummer op te halen
        try:
            fax = tabel1.find('td', {"data-before": "Fax"}).get_text().replace('(', '').replace(')','')
        except:
            fax = None
        # probeer het mailadres op te halen
        try:
            email = tabel1.find('td', {"data-before": "E-mail"}).find('a')['href'].replace('mailto:', '')
        except:
            email = None
        # probeer de website op te halen
        try:
            website = tabel1.find('td', {"data-before": "Internet"}).find('a')['href']
        except:
            website = None
        # verzamel alle informatie in 1 variabele
        data = (provincienaam, baAdres, baPostcode, baPlaats, paAdres, paPostcode, paPlaats, 
                telefoon, fax, email, website)
        # ter controle, print de verzamelde data
        print(data)
        # schrijf de lijst 'data' als een rij naar de csv
        writer.writerow(data)
    # als het opvragen van de pagina niet gelukt is...
    else:
        # print feedback zodat ik kan zien waar het fout ging
        print('https://almanak.overheid.nl'+i+' opvragen is niet gelukt...')
# als je klaar bent met deze for-loop, sluit dan het csv-bestand
f.close()




('Drenthe', 'Westerbrink 1', '9405 BJ', 'Assen', 'Postbus 122', '9400 AC', 'Assen', '0592 36 55 55', '0592 36 57 77', 'post@drenthe.nl', 'http://www.provincie.drenthe.nl')
('Flevoland', 'Visarenddreef 1', '8232 PH', 'Lelystad', 'Postbus 55', '8200 AB', 'Lelystad', '0320 265 265', '0320 26 52 60', 'provincie@flevoland.nl', 'http://www.flevoland.nl')
('Fryslan', 'Tweebaksmarkt 52', '8911 KZ', 'Leeuwarden', 'Postbus 20120', '8900 HM', 'Leeuwarden', '058 292 59 25', '058 292 51 25', 'provincie@fryslan.frl', 'https://www.fryslan.frl')
('Gelderland', 'Markt 11', '6811 CG', 'Arnhem', 'Postbus  9090 ', '6800 GX', 'Arnhem', '0263599999', None, 'provincieloket@gelderland.nl', 'http://www.gelderland.nl')
('Groningen', 'Martinikerkhof 12', '9712 JG', 'Groningen', 'Postbus 610', '9700 AP', 'Groningen', '0503164911', '050 316 49  33', 'info@provinciegroningen.nl', 'http://www.provinciegroningen.nl')
('Limburg', 'Limburglaan 10', '6229 GA', 'Maastricht', 'Postbus 5700', '6202 MA', 'Maastricht', '043 

## 3. Controle

Ter controle importeer ik de zojuist gemaakte csv, en bekijk ik de tabel nog even. 

In [66]:
# lees de csv met pandas (pd), bewaar als variabele df - kort voor dataframe
df = pd.read_csv('190723 contactgegevens provincies.csv')

In [68]:
# bekijk de data
df

,provincie,baAdres,baPostcode,baPlaats,paAdres,paPostcode,paPlaats,telefoon,fax,email,website
0,Drenthe,Westerbrink 1,9405 BJ,Assen,Postbus 122,9400 AC,Assen,0592 36 55 55,0592 36 57 77,post@drenthe.nl,http://www.provincie.drenthe.nl
1,Flevoland,Visarenddreef 1,8232 PH,Lelystad,Postbus 55,8200 AB,Lelystad,0320 265 265,0320 26 52 60,provincie@flevoland.nl,http://www.flevoland.nl
2,Fryslan,Tweebaksmarkt 52,8911 KZ,Leeuwarden,Postbus 20120,8900 HM,Leeuwarden,058 292 59 25,058 292 51 25,provincie@fryslan.frl,https://www.fryslan.frl
3,Gelderland,Markt 11,6811 CG,Arnhem,Postbus 9090,6800 GX,Arnhem,0263599999,NaN,provincieloket@gelderland.nl,http://www.gelderland.nl
4,Groningen,Martinikerkhof 12,9712 JG,Groningen,Postbus 610,9700 AP,Groningen,0503164911,050 316 49 33,info@provinciegroningen.nl,http://www.provinciegroningen.nl
5,Limburg,Limburglaan 10,6229 GA,Maastricht,Postbus 5700,6202 MA,Maastricht,043 389 99 99,043 361 80 99,postbus@prvlimburg.nl,http://www.limburg.nl
6,Noord-Brabant,Brabantlaan 1,5216 TV,'s-Hertogenbosch,Postbus 90151,5200 MC,'s-Hertogenbosch,073 681 28 12,073 614 11 15,info@brabant.nl,http://www.brabant.nl
7,Noord-Holland,Houtplein 33,2012 DE,Haarlem,Postbus 3007,2001 DA,Haarlem,0235143143,NaN,post@noord-holland.nl,http://www.noord-holland.nl
8,Overijssel,Luttenbergstraat 2,8012 EE,Zwolle,Postbus 10078,8000 GB,Zwolle,038 499 88 99,038 425 48 88,postbus@overijssel.nl,http://www.overijssel.nl
9,Utrecht,Pythagoraslaan 101,3584 BB,Utrecht,Postbus 80300,3508 TH,Utrecht,0302589111,0302522564,info@provincie-utrecht.nl,http://www.provincie-utrecht.nl


Als ik wil weten hoeveel provincies geen fax-nummer hebben, dan moet ik daar 3 stappen voor doen: 
1. neem de fax-kolom:      `df['fax']`
2. bekijk per rij of dit een None waarde is of niet:      `.isna()`
3. tel alle none-waarden bij elkaar op:     `.sum()`

In code ziet dat er dus zo uit:

In [72]:
# aantal Nones in fax kolom
df['fax'].isna().sum()

3